In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings

In [2]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings('ignore')

path = os.getcwd()

In [4]:
df = pd.read_csv(path+'/input/df_with_next.csv')
df.head()

,Filename,Altitude,Delta,North,East,path,next_altitude,next_path
0,0a118184382a407bba7aef472932273e.jpg,138.308090,0.069168,0.017955,-0.760026,/home/ec2-user/SageMaker/k_project/input/train...,138.377258,/home/ec2-user/SageMaker/k_project/input/train...
1,1977ab8c9f9473d8594671be4ddf9e7f.jpg,138.377258,0.352982,0.415624,-0.539132,/home/ec2-user/SageMaker/k_project/input/train...,138.730240,/home/ec2-user/SageMaker/k_project/input/train...
2,3817157c9127b4cdb7a8d690ee72d874.jpg,138.513351,-0.205261,-0.311030,-0.910864,/home/ec2-user/SageMaker/k_project/input/train...,138.308090,/home/ec2-user/SageMaker/k_project/input/train...
3,fa5375df32bda8862801ea43b6358683.jpg,138.730240,0.172699,0.700283,-0.106404,/home/ec2-user/SageMaker/k_project/input/train...,138.902939,/home/ec2-user/SageMaker/k_project/input/train...
4,f81e516d45f7143402e9482b47248e3b.jpg,138.755569,-0.242218,-0.445744,-0.832371,/home/ec2-user/SageMaker/k_project/input/train...,138.513351,/home/ec2-user/SageMaker/k_project/input/train...


In [4]:
df['next_path_count'] = df.groupby('next_path')['Filename'].transform('count')

In [5]:
df = df[df.next_path_count == 1].reset_index(drop=True)

In [6]:
first_filenames = df.path.values
next_filenames = df.next_path.values

begin_filenames = df[~df.path.isin(next_filenames)].path.values
end_filenames = df[df.next_path.isnull()].path.values

In [7]:
print(len(begin_filenames))
print(len(end_filenames))

802
0


In [8]:
df['run_id'] = None
df['run_seq_id'] = None

In [9]:
df['next_path'] = df['next_path'].fillna('SeqEnd')

In [ ]:
run_id = 0

for init_f in tqdm(begin_filenames):
    next_f = df.loc[df.path == init_f, 'path'].values[0]
    next_seq_id = df.loc[df.path == init_f, 'run_seq_id'].values[0]

    run_seq_id = 0
    while ((next_f != 'SeqEnd') & (next_seq_id is None)):
        df.loc[df.path == next_f, 'run_id'] = run_id
        df.loc[df.path == next_f, 'run_seq_id'] = run_seq_id
        run_seq_id += 1
        try:
            next_f = df.loc[df.path == next_f, 'next_path'].values[0]
            next_seq_id = df.loc[df.path == next_f, 'run_seq_id'].values[0]
        except:
            next_f = 'SeqEnd'
            next_seq_id = None
    run_id += 1

  6%|▋         | 51/802 [09:47<3:18:53, 15.89s/it]

In [ ]:
df.head(10)

In [ ]:
df[['Filename', 'Altitude', 'Delta', 'North', 'East', 'run_id', 'run_seq_id']]

In [ ]:
df[['Filename', 'Altitude', 'Delta', 'North', 'East', 'run_id', 'run_seq_id']].to_csv(path+'/input/public_with_sequences.csv', index=False)

### Rest of sequence

In [13]:
df = pd.read_csv(path+'/input/df_with_next.csv')
df_sequence = pd.read_csv(path+'/input/public_with_sequences.csv')

In [14]:
df['next_path_count'] = df.groupby('next_path')['Filename'].transform('count')

In [15]:
df_not_seq = df[df.next_path_count > 1]

In [17]:
df_not_seq.head(5)

,Filename,Altitude,Delta,North,East,path,next_altitude,next_path,next_path_count
349,8b49217b4e704d2c40e5908ebd53eda5.jpg,141.177338,0.002945,-0.016418,0.009076,/home/ec2-user/SageMaker/k_project/input/train...,141.180283,/home/ec2-user/SageMaker/k_project/input/train...,2.0
365,92cdc3666b7883ebeed2973e70725bb1.jpg,141.180191,0.002838,-0.017143,0.009482,/home/ec2-user/SageMaker/k_project/input/train...,141.183029,/home/ec2-user/SageMaker/k_project/input/train...,2.0
383,0291fd85422559cef53644838ad97856.jpg,141.182800,-0.003235,-0.015926,0.008802,/home/ec2-user/SageMaker/k_project/input/train...,141.179565,/home/ec2-user/SageMaker/k_project/input/train...,2.0
404,79b76a2914b0e5d228c7ad1ab7e700c2.jpg,141.185776,-0.006210,-0.032545,0.017985,/home/ec2-user/SageMaker/k_project/input/train...,141.179565,/home/ec2-user/SageMaker/k_project/input/train...,2.0
417,95616f3aeb1feb9c4afb8768c0997527.jpg,141.186539,-0.000168,-0.000171,0.000094,/home/ec2-user/SageMaker/k_project/input/train...,141.186371,/home/ec2-user/SageMaker/k_project/input/train...,2.0


In [18]:
next_path_filenames = df_not_seq.next_path.unique()

In [20]:
run_id = df_sequence.run_id.max()
run_id

801.0

In [21]:
for init_f in tqdm(next_path_filenames):
    run_id += 1
    df_not_seq.loc[df_not_seq.next_path == init_f, 'run_id'] = run_id
    seq_filenames = df_not_seq.loc[df_not_seq.next_path == init_f, 'path'].sort_values().values
    run_seq_id = 0
    for seq_file in seq_filenames:
        df_not_seq.loc[df_not_seq.path == seq_file, 'run_seq_id'] = run_seq_id
        run_seq_id += 1

100%|██████████| 480/480 [00:00<00:00, 938.45it/s]


In [23]:
df_not_seq = df_not_seq[['Filename', 'Altitude', 'Delta', 'North', 'East', 'run_id', 'run_seq_id']]

In [25]:
df_sequence_final = pd.concat([df_sequence, df_not_seq])

In [37]:
df_sequence_final.sort_values(by=['run_id', 'run_seq_id'])

,Filename,Altitude,Delta,North,East,run_id,run_seq_id
355,dcf05231c492569270f19d3f1780e475.jpg,141.179565,-0.002914,-0.016980,0.009377,0.0,0.0
340,1c6ae2e3522e87b5f0eba1e226dc5029.jpg,141.176651,0.019150,-0.099345,0.054884,0.0,1.0
450,43542e3a42df9043e6a500e1c2564fc6.jpg,141.195801,-0.022507,-0.115461,0.063855,0.0,2.0
327,8fad6d7e58542a344408f83a0a73e11b.jpg,141.173294,0.000977,-0.000265,0.000146,0.0,3.0
336,9293eb98abb2abe316a3598a83c70514.jpg,141.174271,-0.004089,-0.016188,0.008945,0.0,4.0
...,...,...,...,...,...,...,...
115380,3588e57c66a6dee6c900b58404697749.jpg,205.133942,-0.000519,-0.069345,0.110373,NaN,NaN
119301,7ffc2fa46fbc98fb9794db5e1f7d4896.jpg,207.206390,-0.000244,0.093526,0.276666,NaN,NaN
119357,9ad869837fac24698bc56ead9e0dae24.jpg,207.219910,-0.000244,0.097634,0.265782,NaN,NaN
119376,7f8e971121fea7afbe5297935b4546ea.jpg,207.224152,-0.000275,0.107426,0.281032,NaN,NaN


In [35]:
pb = pd.read_csv(path+'/input/public.csv')

In [36]:
isolated_df = pb[~pb.Filename.isin(df_sequence_final.Filename)].reset_index(drop=True)

In [38]:
isolated_df['run_id'] = np.nan
isolated_df['run_seq_id'] = np.nan

In [39]:
df_sequence_final = pd.concat([df_sequence_final, isolated_df])

In [44]:
df_sequence_final = df_sequence_final.reset_index(drop=True)

In [46]:
left_path_filenames = df_sequence_final[df_sequence_final.run_id.isnull()].Filename.unique()

In [47]:
run_id = df_sequence_final.run_id.max()

In [48]:
for init_f in tqdm(left_path_filenames):
    run_id += 1
    df_sequence_final.loc[df_sequence_final.Filename == init_f, 'run_id'] = run_id
    run_seq_id = 0
    df_sequence_final.loc[df_sequence_final.Filename == init_f, 'run_seq_id'] = run_seq_id

100%|██████████| 381/381 [00:03<00:00, 106.95it/s]


In [49]:
df_sequence_final

,Filename,Altitude,Delta,North,East,run_id,run_seq_id
0,0a118184382a407bba7aef472932273e.jpg,138.308090,0.069168,0.017955,-0.760026,32.0,126.0
1,1977ab8c9f9473d8594671be4ddf9e7f.jpg,138.377258,0.352982,0.415624,-0.539132,32.0,127.0
2,3817157c9127b4cdb7a8d690ee72d874.jpg,138.513351,-0.205261,-0.311030,-0.910864,32.0,125.0
3,fa5375df32bda8862801ea43b6358683.jpg,138.730240,0.172699,0.700283,-0.106404,32.0,128.0
4,f81e516d45f7143402e9482b47248e3b.jpg,138.755569,-0.242218,-0.445744,-0.832371,32.0,124.0
...,...,...,...,...,...,...,...
146257,fc65c23ae61ce2c6fdbc637a8296d1d7.jpg,199.826904,0.232056,NaN,NaN,1658.0,0.0
146258,fc6c0d99ec42bad5e2d5fbb1e71c5a50.jpg,216.407486,0.102585,NaN,NaN,1659.0,0.0
146259,fdbb732633c6a3d32a67fc56edc7cde4.jpg,191.262970,0.053955,NaN,NaN,1660.0,0.0
146260,feac1b902c7487dcbdf298bd4be75690.jpg,210.220703,-0.028854,NaN,NaN,1661.0,0.0


In [50]:
df_sequence_final.to_csv(path+'/input/public_with_sequences_final.csv', index=False)